# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
P. Molliere  ->  P. Molliere  |  ['P. Molliere']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Li  ->  J. Li  |  ['J. Li']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 50 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.09009


extracting tarball to tmp_2407.09009...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.09194


extracting tarball to tmp_2407.09194...

...

extracting tarball to tmp_2407.09312...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.09399


extracting tarball to tmp_2407.09399...

 done.


Found 156 bibliographic references in tmp_2407.09399/main_text.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.09399-b31b1b.svg)](https://arxiv.org/abs/2407.09399) | **Rich and diverse molecular gas environments of closely-separated dual quasars viewed by ALMA**  |
|| S. Tang, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-07-15*|
|*Comments*| **|
|**Abstract**|            We present a study of the molecular gas in five closely-spaced ($R_{\perp}<20$ kpc) dual quasars ($L_{\rm bol}\gtrsim10^{44}~\mathrm{erg~s}^{-1}$) at redshifts $0.4<z<0.8$ with the Atacama Large Millimeter/submillimeter Array. The dual quasar phase represents a distinctive stage during the interaction between two galaxies for investigating quasar fueling and feedback effects on the gas reservoir. The dual quasars were selected from the Sloan Digital Sky Survey and Subaru/Hyper Suprime-Cam Subaru Strategic Program, with confirmatory spectroscopic validation. Based on the detection of the CO J=2--1 emission line with Band 4, we derived key properties including CO luminosities, line widths, and molecular gas masses for these systems. Among the ten quasars of the five pairs, eight have line detections exceeding $5\sigma$. The detected sources prominently harbor substantial molecular gas reservoirs, with molecular gas masses ($M_{\text{molgas}}$) between $10^{9.6-10.5}~\mathrm{M_{\odot}}$, and molecular gas-to-stellar mass ratios ($\mu_{\text{molgas}}$) spanning $18-97\%$. The overall $\mu_{\text{molgas}}$ of these dual quasars agrees with that of inactive star-forming main-sequence galaxies at comparable redshifts, indicating no clear evidence of quenching. However, intriguing features in each individual system show possible evidence of AGN feedback, matter transfer, and compaction processes.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.09009-b31b1b.svg)](https://arxiv.org/abs/2407.09009) | **Probing Cold-to-Temperate Exoplanetary Atmospheres: The Role of Water Condensation on Surface Identification with JWST**  |
|| Z. Huang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-07-15*|
|*Comments*| *21 pages, 7 figures*|
|**Abstract**|            Understanding the surface temperature and interior structure of cold-to-temperate sub-Neptunes is critical for assessing their habitability, yet direct observations are challenging. In this study, we investigate the impact of water condensation on the atmospheric compositions of sub-Neptunes, focusing on the implications for JWST spectroscopic observations. By modeling the atmospheric photochemistry of two canonical sub-Neptunes, K2-18 b and LHS 1140 b, both with and without water condensation and with and without thick atmospheres, we demonstrate that water condensation can significantly affect the predicted atmospheric compositions. This effect is driven by oxygen depletion from the condensation of water vapor and primarily manifests as an increase in the C/O ratio within the photochemically active regions of the atmosphere. This change in composition particularly affects planets with thin H$_2$-dominated atmospheres, leading to a transition in dominant nitrogen and carbon carriers from N$_2$ and oxygen-rich species like CO/CO$_2$ towards heavier hydrocarbons and nitriles. While our models do not fully account for the loss mechanisms of these higher-order species, such molecules can go on to form more refractory molecules or hazes. Planets with thin H2-rich atmospheres undergoing significant water condensation are thus likely to exhibit very hazy atmospheres. The relatively flat JWST spectra observed for LHS 1140 b could be consistent with such a scenario, suggesting a shallow surface with extensive water condensation or a high atmospheric C/O ratio. Conversely, the JWST observations of K2-18 b are better aligned with a volatile-rich mini-Neptune with a thick atmosphere.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.09312-b31b1b.svg)](https://arxiv.org/abs/2407.09312) | **Detailed Mapping of the Galactic Disk Structure in the Solar Neighborhood through LAMOST K Dwarfs**  |
|| X.-C. Tang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-07-15*|
|*Comments*| *15 pages, 24 figures, 6 tables; accepted for publication in MNRAS*|
|**Abstract**|            The Galactic disk is one of the main components of the Milky Way, which contributes most of the luminosity. Its structure is essential for understanding the formation and evolution of the Milky Way. Using 174,443 K-type dwarf stars observed by both LAMOST and Gaia DR3, we study the disk density profile in the local volume within 1,200 pc. In the azimuthal dimension, we find strong asymmetric signal of the thin disk. The surface density and the scale height of the southern disk significantly change versus the azimuthal angle at the same galactocentric distance $R$. Meanwhile, in the vertical dimension, the scale height of the northern disk has quite different trend than that of the southern one. The scale height of the southern disk shows a decreasing trend with $\phi\sim-2.5^\circ$, and change to an increasing one with $\phi\sim5.0^°$. Meanwhile, the scale height of the northern disk has a consistently smaller increase. Finally, we divide the entire sample into three subsamples based on metallicity and all three subsamples show significant non-axisymmetric and north-south asymmetric signals in the Galactic disk. Furthermore, we find that the scale height of the metal-poor ([Fe/H] $<$ -0.4 dex) subsample in the northern disk is greater than that of the metal-rich ([Fe/H] $>$ -0.1 dex) subsample. However, in the southern disk, the scale height exhibits varying relationships across different metallicity slices.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.09194-b31b1b.svg)](https://arxiv.org/abs/2407.09194) | **The JWST Weather Report from the Nearest Brown Dwarfs I: multi-period JWST NIRSpec + MIRI monitoring of the benchmark binary brown dwarf WISE 1049AB**  |
|| B. A. Biller, et al. -- incl., <mark>P. Molliere</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-07-15*|
|*Comments*| *28 pages, 27 figures, accepted to MNRAS*|
|**Abstract**|            We report results from 8 hours of JWST/MIRI LRS spectroscopic monitoring directly followed by 7 hours of JWST/NIRSpec prism spectroscopic monitoring of the benchmark binary brown dwarf WISE 1049AB, the closest, brightest brown dwarfs known. We find water, methane, and CO absorption features in both components, including the 3.3 $\mu$m methane absorption feature and a tentative detection of small grain ($<$ 1$\mu$m) silicate absorption at $>$8.5 $\mu$m in WISE 1049A. Both components vary significantly ($>$1$\%$), with WISE 1049B displaying larger variations than WISE 1049A. Using K-means clustering, we find three main transition points in wavelength for both components of the binary: 1) change in behavior at $\sim$2.3 $\mu$m coincident with a CO absorption bandhead, 2) change in behavior at 4.2 $\mu$m, close to the CO fundamental band at $\lambda >$ 4.4 $\mu$m, and 3) change in behavior at 8.3-8.5 $\mu$m, potentially corresponding to silicate absorption. We interpret the lightcurves observed with both NIRSpec and MIRI as likely stemming from 1) a deep pressure level driving the double-peaked variability seen in WISE 1049B at wavelengths $<$2.3 $\mu$m and $>$8.5 $\mu$m, 2) an intermediate pressure level shaping the lightcurve morphology between 2.3 and 4.2 $\mu$m, and 3) a higher-altitude pressure level producing single-peaked and plateaued lightcurve behavior between 4.2 and 8.5 $\mu$m.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.09399/./z_comparison.png', 'tmp_2407.09399/./084710.40-001302.6_CO_map.png', 'tmp_2407.09399/./121405.12+010205.1_CO_map.png']
copying  tmp_2407.09399/./z_comparison.png to _build/html/
copying  tmp_2407.09399/./084710.40-001302.6_CO_map.png to _build/html/
copying  tmp_2407.09399/./121405.12+010205.1_CO_map.png to _build/html/
exported in  _build/html/2407.09399.md
    + _build/html/tmp_2407.09399/./z_comparison.png
    + _build/html/tmp_2407.09399/./084710.40-001302.6_CO_map.png
    + _build/html/tmp_2407.09399/./121405.12+010205.1_CO_map.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Rich and diverse molecular gas environments of closely-separated dual quasars viewed by ALMA

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.09399-b31b1b.svg)](https://arxiv.org/abs/2407.09399)<mark>Appeared on: 2024-07-15</mark> - 

</div>
<div id="authors">

S. Tang, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We present a study of the molecular gas in five closely-spaced ( $R_{\perp}<20$ kpc) dual quasars ( $L_{\rm bol}\gtrsim10^{44} \mathrm{erg s}^{-1}$ ) at redshifts $0.4<z<0.8$ with the Atacama Large Millimeter/submillimeter Array. The dual quasar phase represents a distinctive stage during the interaction between two galaxies for investigating quasar fueling and feedback effects on the gas reservoir. The dual quasars were selected from the Sloan Digital Sky Survey and Subaru/Hyper Suprime-Cam Subaru Strategic Program, with confirmatory spectroscopic validation. Based on the detection of the CO J=2--1 emission line with Band 4, we derived key properties including CO luminosities, line widths, and molecular gas masses for these systems. Among the ten quasars of the five pairs, eight have line detections exceeding $5\sigma$ . The detected sources prominently harbor substantial molecular gas reservoirs, with molecular gas masses ( $\mmg$ ) between $10^{9.6-10.5} \mathrm{M_{\odot}}$ , and molecular gas-to-stellar mass ratios ( $\umg$ ) spanning $18-97\%$ . The overall $\umg$ of these dual quasars agrees with that of inactive star-forming main-sequence galaxies at comparable redshifts, indicating no clear evidence of quenching. However, intriguing features in each individual system show possible evidence of AGN feedback, matter transfer, and compaction processes.

</div>

<div id="div_fig1">

<img src="tmp_2407.09399/./z_comparison.png" alt="Fig4" width="100%"/>

**Figure 4. -** Left: $\lcotwo$ as a function of redshift for the dual quasars in this study (green stars) and single quasars from the literature (colored dots). Right: Molecular gas to stellar mass ratio ($\umg = \mmg / M_*$) as a function of the redshift, using symbols similar to those in the left panel. Additionally, the figure includes the best-fitting results for the star-forming main sequence (SFMS), starburst (SB), and quiescent (QS) galaxies represented by black, blue, and red solid lines, respectively. The uncertainties associated with $R_{21}$ and $\alpha_{\mathrm{CO}}$ values are displayed at the bottom right. The pair of SDSS J0847-0013 and J1416+0033 are shown as a single data point (the green stars with black edges) because their host galaxies cannot be separated in HSC image, and we only have one $M_*$(thus one $\umg$) measurement for each pair. (*fig:gas_frac*)

</div>
<div id="div_fig2">

<img src="tmp_2407.09399/./084710.40-001302.6_CO_map.png" alt="Fig5" width="100%"/>

**Figure 5. -** CO (J=2--1) properties of SDSS J0847-0013. (a): CO M0 (intensity) map as described in Section \ref{subsec:observation}. The underlying color image is from Subaru/HSC $g$, $r$, and $i$ band data. The white ellipse at the bottom left indicates the beam size of our ALMA observation, and the projected optical separation between the two nuclei is shown as the scale bar at the bottom right with angular and physical scales. (b): CO M1 (velocity) map generated with \texttt{immoment} in unit of km/s. The velocity center is selected to be the center of the Gaussian model of the CO profile in panel (g). (c): CO M2 (dispersion) map generated with \texttt{immoment} in unit of km/s. (d): Same CO M0 map as in panel (a) with the flux level shown by the scale bar. (e): 2D gaussian model of CO M0 map generated from \texttt{imfit}. (f): CO residuals of the M0 map after subtracting the model in panel (e) from the data in Panel (d). (g) and (h): Emission line profiles of the two sources extracted from the regions marked in panel (d) together with the noise spectra extracted from offset regions from the sources, the best-fit model with 3$\sigma$ uncertainties from lmfit, and $W_{50}$ of the emission line. The arrows at the top indicate the expected position of the CO line based on the optical spectroscopic redshift. (*fig:0847_CO_map*)

</div>
<div id="div_fig3">

<img src="tmp_2407.09399/./121405.12+010205.1_CO_map.png" alt="Fig6" width="100%"/>

**Figure 6. -** CO properties of SDSS J1214+0102. The format is similar to Figure \ref{fig:0847_CO_map} (*fig:1214_CO_map*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.09399"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

133  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

19  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
